In [26]:
import boto3
import json
import requests
import pandas as pd
import os
import glob
import json
import concurrent.futures
import requests
from datetime import datetime

s3 = boto3.client('s3')
BUCKET_NAME = 'sagemaker-crawling-s3-bucket'
DIR_NAME = 'data/' #data_500 -> s&p500

In [27]:

headers = {'User-Agent': 'Chrome/111.0.0.0 Safari/537.36'}

data = {'mktsel': 'ALL',
        'typeNo': '0',
        'searchText': '',
        'bld': 'dbms/comm/finder/finder_stkisu'}

url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

r = requests.post(url, data=data, headers=headers)
j = json.loads(r.text)

df = pd.json_normalize(j['block1'])
print(len(df))
print(df.iloc[1]['full_code'])
     

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [23]:
def crawl_stock_data(args):
    full_code, codeName = args
    headers = {'User-Agent': 'Chrome/111.0.0.0 Safari/537.36'}
            
            
    strtDd = '19800101'
    endDd = str(datetime.today().strftime("%Y%m%d"))

    data2 = {'bld': 'dbms/MDC/STAT/standard/MDCSTAT01701',
            'isuCd': full_code,
            'isuCd': full_code,
            'strtDd': strtDd,
            'endDd': endDd,
            'searchType': '2',
            'mktld': 'ALL',
            'csvxls_isNo': 'false'}

    url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    r2 = requests.post(url2, data2, headers=headers)
    j2 = json.loads(r2.text)
    df3 = pd.DataFrame(j2['output'])
    df3['ISU_NM'] = codeName
            

    # 데이터를 CSV로 변환
    csv_data = df3.to_csv(index=False)
            
    # S3에 데이터 업로드
    s3_object_key = f"{full_code}.csv"
    KEY = DIR_NAME + s3_object_key
    s3.put_object(Body=csv_data, Bucket=BUCKET_NAME, Key=KEY)
    print(f'{full_code}')

In [24]:
with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
    # 각 주식 코드에 대한 크롤링 함수를 병렬로 실행
    args_list = zip(df["full_code"], df["codeName"])
    results = list(executor.map(crawl_stock_data, args_list))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)